In [ ]:
import sys
import numpy as np
import pandas as pd

In [ ]:
# import data
from ast import literal_eval
train = pd.read_csv("/home/samyakj/Desktop/Toxic_Span/Dataset/tsd_train.csv")
train["spans"] = train.spans.apply(literal_eval)    


In [ ]:
# corpus = dup of "train"
corpus = train.copy()

#generate toxic chars
toxic = [ '' for i in range(len(corpus))]
for i in range(0, len(corpus)):
    string = corpus.text[i]
    posi = corpus.spans[i]
    for j in range(len(posi)):
        toxic[i] += string[posi[j]]

corpus["toxic_chars"] = toxic
del toxic
  

In [ ]:
# finding white spaces which are marked toxic
def find_whitespace(string):
    count = 0
    for i in string:
        if(i.isspace()):
            count=count+1  
    return count
corpus["toxic_whitespaces"] = corpus.toxic_chars.apply(lambda x: find_whitespace(x))
corpus["total_toxic_char"] = corpus.toxic_chars.apply(lambda x: len(x))
corpus["white_by_char"] = 100*corpus['toxic_whitespaces']/(corpus['total_toxic_char']+.00000001)



In [ ]:
# stats of words and chars
length_of_words = corpus['text'].apply(lambda x: len(x.split()))
length_of_text = corpus['text'].apply(lambda x: len(x))

In [ ]:
#pd.DataFrame({'word_level':length_of_words.describe(), 'char_level': length_of_text.describe()} )

In [ ]:
# tokenization and cacheing spans(position of token)
from nltk.tokenize import TreebankWordTokenizer as twt

def tokens(string):
    tokens = twt().tokenize(string)
    return tokens 

def spans_of_tokens(string):
    list_of_spans = list(twt().span_tokenize(string))
    return list_of_spans

corpus["tokenize"] = corpus["text"].apply(lambda x: tokens(x))
corpus["spans_of_tokens"] = corpus["text"].apply(lambda x: spans_of_tokens(x))

In [ ]:
# target 0, 1 notation

def target_vector_(span, span_of_token):
    target = []
    for x,y in span_of_token:
            if x in span:
                target.append(1)
            else:
                target.append(0)
        
    return target

temp = []
for i in range(len(corpus)):
        temp.append(target_vector_(corpus.spans[i], corpus.spans_of_tokens[i]))
        
corpus["target"] = temp


In [ ]:
# zipped together -> token, their spans, and target
temp = []
for i in range(len(corpus)):
    temp.append(list(zip(corpus.tokenize[i], corpus.spans_of_tokens[i], corpus.target[i])))

for i in range(len(temp)):    
    temp[i] = [list(ele) for ele in temp[i]]  

corpus["zipped"] = temp
del temp


In [ ]:
#################### cleaning functions ####################
# emoji removing function
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

#puntuation remover
from string import punctuation
def strip_punctuation(text):
    return ''.join(c for c in text if c not in punctuation)

#convert accented text
import unicodedata
def convert_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

#expand contraction
from contractions import CONTRACTION_MAP # from contractions.py
import re 

# function to expand contractions
def expand_contractions(text, map=CONTRACTION_MAP):
    if text in CONTRACTION_MAP.keys():
        return CONTRACTION_MAP[text]
    else:
        return text

# function to remove digits
def remove_nums(text):
    return re.sub('[0-9]+', '', text)

# remove special chars
def remove_spe_chars(text):
    return re.sub('[\W_]+', '', text)
#################### cleaning functions ####################    

In [ ]:
#cleaning tokens (in zipped)
def clean_string(string):
    string = string.lower()#lowercasing
    string = expand_contractions(string) #expand contraction
    string = remove_nums(string)#remove digits
    #numbers into words(could be done if required)# not require though
    string = convert_accented_chars(string)#convert accented
    string = deEmojify(string)#remove emoji
    string = remove_spe_chars(string)#remove all puntuation
    
    return string

def cleaning(iterable):
    for i in range(len(iterable)):
        iterable[i][0] = clean_string(iterable[i][0])    
    
    return iterable


In [ ]:
#remove empty tokens (in zipped)
def remove_empty(iterable):
    iterable = list(filter(lambda x: x[0] != '', iterable))
    return iterable

In [ ]:
# clean
corpus.zipped = corpus.zipped.apply(lambda x: cleaning(x))
# remove nulls
corpus.zipped = corpus.zipped.apply(lambda x: remove_empty(x))

In [ ]:
#corpus.to_csv('processed.csv', index = False)

In [ ]:
def reddem_target(lis):
    arr = []
    for item in lis:
        arr.append(item[2])
    return arr

def redeem_spans(lis):
    arr = []
    for item in lis:
        arr.append(item[1])
    return arr

def reddem_token(lis):
    arr = []
    for item in lis:
        arr.append(item[0])
    return arr

In [ ]:
corpus['target_final'] = corpus.zipped.apply(lambda x : reddem_target(x))

corpus['span_final'] = corpus.zipped.apply(lambda x : redeem_spans(x))

corpus['token_final'] = corpus.zipped.apply(lambda x : reddem_token(x))

In [ ]:
final_data = corpus[['spans', 'text', 'target_final', 'span_final', 'token_final']].copy()

In [ ]:
final_data.to_pickle("./final.pkl")

In [ ]:
final_data.to_csv("final.csv")